# Section 1.5  Model comparison of GP (R)

In [1]:
# Platform information
library(benchmarkme)
get_platform_info()$OS.type
get_r_version()$version.string
get_cpu()$model_name;get_cpu()$no_of_cores
get_ram()

[1] "windows"

[1] "R version 3.5.1 (2018-07-02)"

[1] "Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz"

[1] 8

8.59 GB

In [1]:
# Parallel computation
library(doParallel)
library(foreach)
cl<-makeCluster(8) 

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


In [2]:
# Loading data
geno <-read.table("./Hetero_realigned_cov10_filtered3.raw", row.names=1, header=T)
row.names(geno)=1:240
x <- as.matrix(geno)-1   
pheno <- read.csv("./2017heteroPheno.csv", header=T)
attach(pheno)
data <- data.frame(tbcw=sqrt(pheno$bcw+1),bcw=pheno$bcw,length=pheno$length,gid=1:240)

In [3]:
# Parameters for cross validation
repeats <- 10
n.fold <- 5 
n.sample <- length(pheno$bcw)    
CM<-7

## GBLUP

In [4]:
# Package
library(rrBLUP)
packageVersion("rrBLUP")

[1] '4.6'

In [5]:
# Marker-based relationship matrix (Endelman at al. 2011)
A <- A.mat(x, n.core=8)
row.names(A)=1:240;colnames(A)=1:240

In [6]:
registerDoParallel(cl)
system.time({
GBLUP<-foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="rrBLUP") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         res <- kin.blup(bcw_test, K=A, geno="gid", pheno="tbcw")
         cor(data$tbcw[id==i],res$pred[id==i])
    }
   }
})
stopImplicitCluster()

   user  system elapsed 
   3.75    0.44    1.09 

# Bayes A

In [8]:
# Package
library("BGLR")
packageVersion("BGLR")

[1] '1.0.5'

In [ ]:
registerDoParallel(cl)
system.time({
BA <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         fmBA=BGLR(y=bcw_test$tbcw,ETA=list(list(X=x,model='BayesA')),nIter=2000,burnIn=1000)
         cor(data$tbcw[id == i],fmBA$yHat[id == i])
         }  
    }
})
stopImplicitCluster()  

# Bayes B

In [ ]:
registerDoParallel(cl)
system.time({
BB <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         fmBB=BGLR(y=bcw_test$tbcw,ETA=list(list(X=x,model='BayesB')),nIter=2000,burnIn=1000)
         cor(data$tbcw[id == i],fmBB$yHat[id == i])
         }  
    }
})
stopImplicitCluster()  

# Bayes C

In [ ]:
registerDoParallel(cl)
system.time({
BC <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         fmBC=BGLR(y=bcw_test$tbcw,ETA=list(list(X=x,model='BayesC')),nIter=2000,burnIn=1000)
         cor(data$tbcw[id == i],fmBC$yHat[id == i])
         }  
    }
})
stopImplicitCluster()  

# Baces BL

In [ ]:
registerDoParallel(cl)
system.time({
BL <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         fmBL=BGLR(y=bcw_test$tbcw,ETA=list(list(X=x,model='BL')),nIter=2000,burnIn=1000)
         cor(data$tbcw[id == i],fmBL$yHat[id == i])
         }  
    }
})
stopImplicitCluster()  

## Bayes RKHS

In [11]:
# Reproducing kernel
p<-ncol(x)
D<-(as.matrix(dist(x,method='euclidean'))^2)/p
h<-0.5;K<-exp(-h*D)
ETA<-list(list(K=K,model='RKHS'))

In [ ]:
registerDoParallel(cl)
system.time({
RKHS <- foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %do% {
         bcw_test <- data
         bcw_test$bcw[id == i] <- NA
         fmRKHS=BGLR(y=bcw_test$bcw,ETA=ETA,nIter=2000,burnIn=1000)
         cor(pheno$bcw[id == i],fmRKHS$yHat[id == i])
         }    
      }
})
stopImplicitCluster()  

## Comparision 

In [16]:
#Save result
Acc<-data.frame(unlist(GBLUP),unlist(BA), unlist(BB),unlist(BC),unlist(BL),unlist(RKHS))
colnames(Acc)<-c("GBLUP","BA","BB","BC","BL","RKHS")
library("xlsx")
write.xlsx(Acc, "All_models_Acc.xlsx") 

In [20]:
#Load data(from this page and deep learning models)
library("readxl")
data<-read_excel("All_models_Acc.xlsx")
Acc_all<-subset(data,select = - c(X__1))

In [21]:
(summary<-data.frame(Acc_mean=sapply(Acc_all,function(x) round(mean(x),digits = 3)),
    Acc_SE=sapply(Acc_all,function(x) round(sd(x)/sqrt(repeats*n.fold),digits = 3))))

,Acc_mean,Acc_SE
GBLUP,0.284,0.017
BA,0.279,0.017
BB,0.288,0.016
BC,0.284,0.016
BL,0.290,0.016
RKHS,0.280,0.017


In [22]:
write.xlsx(summary, "1.7 summary.xlsx") 